In [2]:
import openai 
client = openai.Client()



In [3]:
vector_store = client.beta.vector_stores.create(name="Apostilas Asimov Aula 15")

In [13]:
files = ['openAI.pdf', 'hugging-face.pdf']

file_steam = [open(f, 'rb') for f in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
	vector_store_id = vector_store.id,
	files=file_steam
)

In [15]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [18]:
assistant = client.beta.assistants.create(
    name="Tutor Asimov",
    instructions="Você é um tutor de uma escola de programção. Você é ótimo para responder \
        perguntas teóricas sobre a api da OpenAI e sobre a utlização da biblioteca do Hugging \
		Face com python. Você utiliza as apostilas dos cursos para basear sua respostas. Caso você não \
		encontre as respostas nas apostilas informadas, você fala que não saber responder.",
    tools=[{"type": "file_search"}],
    tool_resources={'file_search': {"vector_store_ids": [vector_store.id]}},
    model="gpt-3.5-turbo",
)

In [19]:
thread = client.beta.threads.create()

In [20]:
texto_mensagem = "Segundo o documento fornecido, o que é o Hugging Face?"
message = client.beta.threads.messages.create(
	thread_id=thread.id,
 	role="user",
	content=texto_mensagem
)

In [21]:
run = client.beta.threads.runs.create(
	thread_id=thread.id,
	assistant_id=assistant.id,
	instructions="O nome do usuário é Adriano Soares e ele é um usuário Premium."
)

In [22]:
import time 

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
		thread_id=thread.id,
		run_id=run.id
	)

In [27]:
run.status

'completed'

In [28]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
		thread_id=thread.id
	)
    print(mensagens)

else:
    print('Error')

SyncCursorPage[Message](data=[Message(id='msg_F61nCqG2s7rCHwbSAfWTk005', assistant_id='asst_mvXtvzMTM6g7cOKP1VbT7ERp', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=297, file_citation=FileCitation(file_id='file-sLH3q9D9Xg4fR60djuFWFhGb'), start_index=285, text='【4:0†source】', type='file_citation')], value='O Hugging Face é uma empresa fundada em 2017 na França, inicialmente focada no desenvolvimento de Chatbots. Ao longo do tempo, a empresa passou a desenvolver uma infraestrutura própria e bibliotecas de Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP)【4:0†source】.'), type='text')], created_at=1721419749, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_4wj2dM1zDE45uBLN0yw9uL0n', status=None, thread_id='thread_l3L8maijFX3dxTqHrpcaz2b3'), Message(id='msg_AEC0tBX3vo6NWZVVZLP0goNB', assistant_id=None, attachments=[], comp

In [29]:
mensagens.data[0].content[0].text.value

'O Hugging Face é uma empresa fundada em 2017 na França, inicialmente focada no desenvolvimento de Chatbots. Ao longo do tempo, a empresa passou a desenvolver uma infraestrutura própria e bibliotecas de Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP)【4:0†source】.'

In [36]:
run_steps = client.beta.threads.runs.steps.list(
	thread_id=thread.id,
	run_id=run.id 
)

In [37]:
run_steps
run_steps.data[0]

RunStep(id='step_rFfoxl9lAPyY80NdNSxnpRhF', assistant_id='asst_mvXtvzMTM6g7cOKP1VbT7ERp', cancelled_at=None, completed_at=1721419750, created_at=1721419749, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_4wj2dM1zDE45uBLN0yw9uL0n', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_F61nCqG2s7rCHwbSAfWTk005'), type='message_creation'), thread_id='thread_l3L8maijFX3dxTqHrpcaz2b3', type='message_creation', usage=Usage(completion_tokens=82, prompt_tokens=4711, total_tokens=4793), expires_at=None)

In [ ]:
for step in run_steps.data:
    print('=== step: ', step.step_details.type)
    
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call == 'file_search' :
            	print(tool_call)
            else:
                print('----')
                print(tool_call.code_interpreter.input)
				print('----')
				print('Result')
				print(tool_call.c)

In [58]:
mensagens = client.beta.threads.messages.list(
	thread_id=thread.id
)

mensagem = list(mensagens)[0].content[0].text 

anotacoes = mensagem.annotations 

citacoes = []

for index, anotacao in enumerate(anotacoes):
    mensagem.value = mensagem.value.replace(anotacao.text, f"[{index}]")
    if file_cit := getattr(anotacao, 'file_citation', None):
	    file = client.files.retrieve(file_cit.file_id)
	    citacoes.append(f"[{index}] {file.filename}")
	    # print(file_cit)
mensagem.value
    

'O Hugging Face é uma empresa fundada em 2017 na França, inicialmente focada no desenvolvimento de Chatbots. Ao longo do tempo, a empresa passou a desenvolver uma infraestrutura própria e bibliotecas de Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP)[0].'

In [59]:
citacoes

['[0] hugging-face.pdf']

In [60]:
mensagem.value

'O Hugging Face é uma empresa fundada em 2017 na França, inicialmente focada no desenvolvimento de Chatbots. Ao longo do tempo, a empresa passou a desenvolver uma infraestrutura própria e bibliotecas de Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP)[0].'